In [10]:
import pandas as pd
import numpy as np
import re

In [3]:
features = pd.read_excel('Healthcare_dataset.xlsx', engine='openpyxl', sheet_name='Feature Description')
data = pd.read_excel('Healthcare_dataset.xlsx', engine='openpyxl', sheet_name='Dataset', na_values=['Unknown', 'Other/Unknown'])

In [4]:
idn = [i for i in data.columns if re.match("^idn*", i.lower()) != None]
features.loc[features["Variable"] == 'IDN Indicator']

,Bucket,Variable,Variable Description
7,NaN,IDN Indicator,Flag indicating patients mapped to IDN


In [5]:
# Counting rows with NA
na_count = data.shape[0] - data.dropna().shape[0]
percent_na = na_count / data.shape[0] * 100
print("Percent of rows with NAs = ", percent_na, "%")

Percent of rows with NAs =  69.50934579439252 %


In [6]:
#Extract rows with NA values only
nan_df = data[data.isna().any(axis=1)]
#Count frequency of persistency flag
persist_na = nan_df['Persistency_Flag'].value_counts()['Persistent']
persist_tot = data['Persistency_Flag'].value_counts()['Persistent']

print('How many persistent flags will be removed by deletion method: ', persist_na / persist_tot * 100, '%')

How many persistent flags will be removed by deletion method:  64.70131885182312 %


In [32]:
#Map values to numbers
columns_map = {}
columns_decode = {}
for col in data.columns:
    if str(data[col].dtype) == 'object':  #If it's a categorical variable
        val_map = {}
        val_decode = {}
        mapped_value = 0
        for idx, val in enumerate(data[col].unique()): 
            not_na = True
            if type(val) == float:
                not_na = (np.isnan(val) != True)
            if  not_na == True: #If the value is not NaN add it to the mapping
                val_map[val] = mapped_value 
                val_decode[mapped_value] = val
                mapped_value += 1
            columns_map[col] = val_map
            columns_decode[col] = val_decode
            
print(columns_map['Ethnicity'])

{'Not Hispanic': 0, 'Hispanic': 1}


# OLD Imputation

In [4]:
#dealing with missing data
from sklearn.impute import KNNImputer

df = data.copy()
#Using Mapping to encode data
for col in df.columns:
    if str(df[col].dtype) == 'object':
        df[col] = df[col].map(columns_map[col])
   

imputer= KNNImputer(n_neighbors=5)
df_filled = df.copy()

for col in df_filled.columns:
    dummy = df_filled.copy()
    #If that column has any missing values
    if df_filled[col].isna().any() == True:
        #Get indexes of missing values
        idx_missing_before = list(df_filled[col][df_filled[col].isna()].index)
        #Drop Na from other columns to be use KNN imputer
        dummy.dropna(subset=dummy.columns.difference([col]), inplace=True)
        
        if dummy[col].isna().any() == True: 
            #Get indexes of missing values after dropping rows above
            idx_missing_after = list(dummy[col][dummy[col].isna()].index)
            #Impute missing values and subtitute them in df_filled           
            dummy = imputer.fit_transform(dummy)  
            dummy = pd.DataFrame(dummy, columns=df_filled.columns).apply(round)  #Convert array from imputer() to pandas
            df_filled[col][idx_missing_before] = dummy[col][idx_missing_after]

<ipython-input-4-b141db2e5968>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filled[col][idx_missing_before] = dummy[col][idx_missing_after]


# NEW Imputation

In [54]:
#dealing with missing data
from sklearn.impute import KNNImputer

df = data.copy()
#Using Mapping to encode data
for col in df.columns:
    if str(df[col].dtype) == 'object':
        df[col] = df[col].map(columns_map[col])
   
df_filled = df.copy()
imputer= KNNImputer(n_neighbors=5)
df_filled = imputer.fit_transform(df_filled)
df_filled = pd.DataFrame(df_filled, columns=df.columns).apply(round)
df_filled.isna().any().any()

False

# Outlier detection

In [ ]:

#New dataframe without outliers.
new_df = df_filled.copy()

#Get outliers for every column; outliers are values less than 10% in count
outliers = {}
n_rows = len(data)
for col in new_df.columns[2:]:
    #outliers[col] = list()
    for val in new_df[col].unique():
        first = True
        ratio = dict(new_df[col].value_counts())[val] * 100 / n_rows
        if ratio < 10:
            if first == True:
                outliers[col] = [val]
                first = False
            else:
                outliers[col].append(val)

#for every outlier, if removing it results in more than a 5% deduction in "Persistent" flag, don't remove it
persist_mapping = columns_map['Persistency_Flag']['Persistent']
for i in outliers.keys():
    for j in range(0, len(outliers[i])):
        #Count of persistent values before removing outliers
        old_count = dict(df_filled['Persistency_Flag'].value_counts())[persist_mapping]
        new_count = dict(df_filled.loc[df_filled[i] != outliers[i][j], 'Persistency_Flag'].value_counts())[persist_mapping]
        ratio = new_count * 100 / old_count
        if ratio < 94:
            print(ratio, " ", i, " ", j)

# Descriptive Analysis


In [55]:
#Reverse mapping for visual analysis(decode)
for col in df_filled.columns:
    if col in columns_decode.keys():
        df_filled[col] = df_filled[col].map(columns_decode[col])
#Filter data variables related to comorbidity
df_filled.filter(regex='^Comorb*')

,Comorb_Encounter_For_Screening_For_Malignant_Neoplasms,Comorb_Encounter_For_Immunization,"Comorb_Encntr_For_General_Exam_W_O_Complaint,_Susp_Or_Reprtd_Dx",Comorb_Vitamin_D_Deficiency,Comorb_Other_Joint_Disorder_Not_Elsewhere_Classified,Comorb_Encntr_For_Oth_Sp_Exam_W_O_Complaint_Suspected_Or_Reprtd_Dx,Comorb_Long_Term_Current_Drug_Therapy,Comorb_Dorsalgia,Comorb_Personal_History_Of_Other_Diseases_And_Conditions,Comorb_Other_Disorders_Of_Bone_Density_And_Structure,Comorb_Disorders_of_lipoprotein_metabolism_and_other_lipidemias,Comorb_Osteoporosis_without_current_pathological_fracture,Comorb_Personal_history_of_malignant_neoplasm,Comorb_Gastro_esophageal_reflux_disease
0,N,Y,Y,N,N,Y,N,Y,Y,N,N,N,N,N
1,N,N,Y,N,N,N,N,N,N,N,N,N,N,N
2,Y,N,Y,N,N,N,N,N,N,N,N,N,N,N
3,N,Y,Y,N,Y,N,N,Y,N,N,Y,N,N,Y
4,Y,Y,Y,N,N,N,N,Y,Y,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3419,Y,N,Y,Y,N,N,N,Y,N,Y,Y,N,N,Y
3420,N,N,N,N,N,N,N,N,N,N,N,N,N,N
3421,Y,N,N,Y,Y,N,N,N,N,N,Y,N,N,N
3422,Y,N,N,N,N,N,N,N,N,N,Y,N,N,N


In [53]:
df_filled.head()

,Ptid,Persistency_Flag,Gender,Race,Ethnicity,Region,Age_Bucket,Ntm_Speciality,Ntm_Specialist_Flag,Ntm_Speciality_Bucket,...,Risk_Family_History_Of_Osteoporosis,Risk_Low_Calcium_Intake,Risk_Vitamin_D_Insufficiency,Risk_Poor_Health_Frailty,Risk_Excessive_Thinness,Risk_Hysterectomy_Oophorectomy,Risk_Estrogen_Deficiency,Risk_Immobilization,Risk_Recurring_Falls,Count_Of_Risks
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


# Important Terminology
## IDN 

Stands for **"Integrated Delivery Methods".**<br>
Think Mayo Clinic, basically it's an organization or a project, that involves more than one aspect of the health care business to ensure efficiency; e.g. health care, health insurance, clinics..etc, all in one. 

## HCP

Stands for **Health care provider**.<br>
All paid and unpaid persons serving in healthcare settings who have the potential for direct or indirect exposure to patients or infectious materials. Think Nurses, Physicians...etc.

## NTM

Stands for **Nontuberculous Mycobacteria** Infections. These are basically pathogens, think **COVID-19**.

### Relevant info

NTM are opportunistic pathogens placing some groups at **increased risk**, including those with underlying lung disease or depressed immune systems.<br>
NTM are **environmental organisms** that can be found in soil, dust, and water including natural water sources and municipal water sources.

## Rx

A medical prescription. The symbol "Rx" is usually said to stand for the Latin word "recipe" meaning **"to take."**<br>
**rxdate:** My guess, it is the date up to which the Rx is valid for.

**T-score:** Standard deviation from median under a T-distribution.

## Multiple Risk Factors

## Dexa Scan

It's a bone density scan. Results are compared to the average adult of same age using T-score.<br>
Used to asses body fat loss, as NTM causes loss of appetite and weight loss.

## Fragility Fracture

**WHO:** "Equivalent to a fall from a standing height or less".

## Comorbidity

The simultaneous presence of two or more diseases or medical conditions in a patient.

## Concomitant drug

Concomitant drugs are two or more drugs used or given at or almost at the same time (one after the other, on the same day, etc.).

# What to Expect: Using Data Analytics

From [Improving patient adherence through data-driven insights](https://www.mckinsey.com/industries/pharmaceuticals-and-medical-products/our-insights/improving-patient-adherence-through-data-driven-insights)

There are three principle measures of adherence:

* **Persistence**. How long patients take a drug before either switching to a new drug or stopping treatment entirely. 
* **Compliance**. How closely patients follow the prescribed treatment plan. This is measured by how many persistent patients fill their prescribed doses on schedule, based on the approved product label. 
* **Adherence**. Combined view of compliance and persistence, measured by the share of all patients, who fill their prescribed doses on schedule, based on the approved product label.<br><br>

* Adherence varies from one disease to the other. *(What other diseases was the patient suffering from? In dataset)*
* Low adherence is typical to chronic diseases.*(May expect low adherence from data)*
* Wide variations of persistence levels medication type. *(How was the medicine administered?)*
* Brand of drug (its effectiveness) is a pivoting factor. 

> These findings highlight the importance of identifying specific drivers of adherence for each individual medication or brand—indicating that it is important to understand the specifics of each medication or brand beyond just understanding the broad factors related to the disease category.

## Factors that influence adherence

<img src="https://www.mckinsey.com/~/media/McKinsey/Industries/Pharmaceuticals%20and%20Medical%20Products/Our%20Insights/Improving%20patient%20adherence%20through%20data%20driven%20insights/SVGZ-Improving-patient-adherence-Ex4.svgz" width=500><br>
<br>

**Important:**  

> Adherence may also be affected by factors linked to the healthcare system, such as the expertise, experience, and **quality of the physicians** and the structure of their practices. Across several different diseases, we found that patient adherence varied greatly by the physicians who treated them.

Dataset categorizes the physicians !<br>


**Important:**<br>
> In addition, the study showed that **comorbidities—conditions** that accompany a disease—tend to negatively correlate with adherence.

**Important:**
> The sheer number of medications a patient is taking can also be an important factor

**Important:**

> Recurring trends across diseases show that a patient’s level of involvement in the community and family-related factors are significant influencers.

# Week 8

## Dealing with outliers and NAs: Thought process

[Guide link](https://www.analyticsvidhya.com/blog/2016/01/guide-data-exploration/#)

1. Is the missing data completely random ? Then use deletion; otherwise deletion might cause bias in the model.
2. Imputing (Probably using mode value because we have categorical variables)
3. Use Prediction models (Train the model with a part of the dataset with no missing values.), e.g. Logistic regression..etc
4. KNN Imputation.

[KNN Imputing in Python.](https://www.analyticsvidhya.com/blog/2020/07/knnimputer-a-robust-way-to-impute-missing-values-using-scikit-learn/)